In [1]:
import pandas as pd
import numpy as np

import os 
import xlrd
from os import mkdir

import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows

import shutil

from workalendar.america import Mexico
from datetime import datetime, timedelta
import datetime as dt
import calendar
import time

import unicodedata

import requests

In [2]:
t_inicio = time.time()

## Definición de funciones para el análisis

In [3]:
def API_Banxico(series, inicio, fin):
    
    response = requests.get( f'https://www.banxico.org.mx/SieAPIRest/service/v1/series/{series}/datos/{inicio}/{fin}', 
                            headers = {'Bmx-Token': 'e3980208bf01ec653aba9aee3c2d6f70f6ae8b066d2545e379b9e0ef92e9de25'})

    if response.status_code == 200:
        data = response.json()
        datos_series = data['bmx']['series'][0]['datos']
        df = pd.DataFrame(datos_series)#.replace({error: np.nan})
        df.rename(columns = {'fecha': 'FECHA', 'dato': 'TASA_INTERES'}, inplace = True)
        df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst = True)
        df['TASA_INTERES'] = pd.to_numeric(df['TASA_INTERES'], errors = 'coerce')
        return df
        
    else:
        return print(f'Error al consultar la API: {response.status_code} - {response.text}')

Se realiza el cálculo de la variación diaria ${\Delta_i}$ del valor de las tasas de referencia, donde:


\begin{equation} 
\Delta_i = \text{Rate}_t - \text{Rate}_{t-i}, \quad \forall i \in \mathbb{N}, \, 1 \leq i \leq 5
\end{equation}

In [4]:
def delta_i_rate(data, col_tasa):
    data = data.sort_values('FECHA')
    for i in range(1, 6):
        data[f'DELTA_{i}'] = data[col_tasa].shift(i) - data[col_tasa]
    return data

In [5]:
def resaltar_valores(var, umbrales):
    """"
    Resalta los valores segun los umbrales definidos
    
    Parámetros:
    - var: variación máxima entre cada periodo, tasa/sobretasa y plazo segun corresponda
    - umbrales: diccionario de umbrales y colores {'umbral1': 'color1', 'umbral2': 'color2', 'umbral3': 'color3'}
    """
    
    if not isinstance(var, (float,int)):
        return '' # Se mantiene sin formato para valores que no sean número
    
    for umbral, color in umbrales.items():
        if var > float(umbral):
            return f'background-color: {color}'
    return '' # Se mantiene sin formato para valores que no sean número

In [6]:
def aplicar_resaltado(df, delta_columns, umbrales):
    """"
    Aplica la funcion resaltar_valores a un DataFrame solo a las columnaas deseadas
    
    Parámetros:
    - df: DataFrame
    - delta_columns: Columnas del df a las que se desea aplicar la función resaltar_valores
    - umbrales: diccionario de umbrales y colores {'umbral1': 'color1', 'umbral2': 'color2', 'umbral3': 'color3'}
    """
    
    return df.style.applymap(lambda var: resaltar_valores(var, umbrales), subset = delta_columns)

In [7]:
def formato_condicional_umbrales(tasa_ref):
    formato_columnas = {
        col: '{:.2%}' for col in tasa_ref.columns if col.startswith('DELTA_')
    }
    formato_columnas.update({
        col: '{:%Y-%m-%d}' for col in tasa_ref.columns if col.startswith('FECHA_MAX_')
    })
    return formato_columnas

In [8]:
def periodo_continuo(tasa_ref_bbdd, columna_valor):
    ultimo_vacio = tasa_ref_bbdd[tasa_ref_bbdd[columna_valor].isna()].last_valid_index()
    if ultimo_vacio is not None:
        return tasa_ref_bbdd.iloc[ultimo_vacio + 1:]
    else:
        return tasa_ref_bbdd

### Definición de parámetros para obtención de información desde API's

In [9]:
fecha_inicio = dt.datetime(1996,1,1).strftime('%Y-%m-%d')
fecha_fin = dt.datetime.today().strftime('%Y-%m-%d')

series_tiief1d = 'SF331451'
series_tiie28d = 'SF43783'
series_cetes28d = 'SF43936'

## Sobretasas

+ BONDESD
+ BONDESF
+ SOBRETASA IM (MENSUAL)
+ SOBRETASA IQ (TRIMESTRAL)
+ SOBRETASA IS (SEMESTRAL)

In [10]:
# Se leen los insumos de manera individual y se agrega como nueva clasificación el tipo de sobretasa
sbts_bd = pd.read_csv(r'C:\Users\pmendoza\Downloads\SobretasaBondesD.csv'); sbts_bd['TIPO'] = 'BONDESD'
sbts_bf = pd.read_csv(r'C:\Users\pmendoza\Downloads\SobretasaBondesF.csv'); sbts_bf['TIPO'] = 'BONDESF'
sbts_IM = pd.read_csv(r'C:\Users\pmendoza\Downloads\SobretasaIM.csv'); sbts_IM['TIPO'] = 'IM'
sbts_IQ = pd.read_csv(r'C:\Users\pmendoza\Downloads\SobretasaIQ.csv'); sbts_IQ['TIPO'] = 'IQ'
sbts_IS = pd.read_csv(r'C:\Users\pmendoza\Downloads\SobretasaIS.csv'); sbts_IS['TIPO'] = 'IS'

#Se agrupa en un mismo df todas las sobretasas de interés, se homologan las columnas y se les da el formato correspondiente
sobretasas = pd.concat([sbts_bd, sbts_bf, sbts_IM, sbts_IQ, sbts_IS]).rename(columns = {'VALOR': 'SOBRETASA'})
sobretasas['FECHA'] = pd.to_datetime(sobretasas['FECHA'])
sobretasas

,FECHA,PLAZO,SOBRETASA,TIPO
0,2021-10-04,1,0.000000,BONDESD
1,2021-10-04,28,0.000000,BONDESD
2,2021-10-04,91,0.019327,BONDESD
3,2021-10-04,182,0.047245,BONDESD
4,2021-10-04,360,0.101853,BONDESD
...,...,...,...,...
11479,2025-01-24,360,0.061143,IS
11480,2025-01-24,720,0.115500,IS
11481,2025-01-24,1080,0.075538,IS
11482,2025-01-24,1440,-0.224971,IS


In [11]:
sbts = []

# Se aplica la variación diaria de tasa a tiempo t - tasa a tiwmpo t -i desde i = 1 hasta i = 5
# para cada plazo y tipo sobretasa
for (sobretasa, plazo), grupo in sobretasas.groupby(['TIPO', 'PLAZO']):
    sbts.append(delta_i_rate(grupo.copy(), col_tasa = 'SOBRETASA'))    
sbts = pd.concat(sbts)
display(sbts)


# Se obtienen las máximas variaciones diarias
delta_columns = [col for col in sbts.columns if 'DELTA' in col]
sbts_max = sbts.groupby(['TIPO', 'PLAZO'])[delta_columns].max().reset_index()
display(sbts_max)


# Se obtienen las fechas correspondientes a las máximas variaciones diarias
sbts_max_fechas = sbts.groupby(['TIPO', 'PLAZO']).apply(
    lambda grupo: pd.Series({
        f'FECHA_MAX_{col}': grupo.loc[grupo[col].idxmax(), 'FECHA'] if not grupo[col].isna().all() else None
        for col in delta_columns
    })).reset_index()
display(sbts_max_fechas)


# Se crea un resumen con las máximas variaciones diarias, para cada plazo y sobretasa con sus respectivas fechas 
maximos_sobretasas = pd.merge(sbts_max, sbts_max_fechas, on = ['TIPO', 'PLAZO'])
maximos_sobretasas

,FECHA,PLAZO,SOBRETASA,TIPO,DELTA_1,DELTA_2,DELTA_3,DELTA_4,DELTA_5
0,2021-10-04,1,0.000000,BONDESD,NaN,NaN,NaN,NaN,NaN
9,2021-10-05,1,0.000000,BONDESD,0.000000,NaN,NaN,NaN,NaN
18,2021-10-06,1,0.000000,BONDESD,0.000000,0.000000,NaN,NaN,NaN
27,2021-10-07,1,0.000000,BONDESD,0.000000,0.000000,0.000000,NaN,NaN
36,2021-10-08,1,0.000000,BONDESD,0.000000,0.000000,0.000000,0.000000,NaN
...,...,...,...,...,...,...,...,...,...
11447,2025-01-20,1800,0.085014,IS,0.000783,0.000192,-0.000398,0.003099,0.005266
11456,2025-01-21,1800,0.085632,IS,-0.000618,0.000165,-0.000426,-0.001016,0.002481
11465,2025-01-22,1800,0.086250,IS,-0.000618,-0.001236,-0.000453,-0.001044,-0.001635
11474,2025-01-23,1800,0.086868,IS,-0.000618,-0.001236,-0.001854,-0.001071,-0.001662


,TIPO,PLAZO,DELTA_1,DELTA_2,DELTA_3,DELTA_4,DELTA_5
0,BONDESD,1,0.115000,0.115000,0.115000,0.115000,0.115000
1,BONDESD,28,0.117500,0.117500,0.117500,0.117500,0.116500
2,BONDESD,91,0.012381,0.012619,0.012500,0.012143,0.014310
3,BONDESD,182,0.008316,0.010000,0.010000,0.010000,0.014000
4,BONDESD,360,0.109829,0.108922,0.108425,0.108075,0.107724
5,BONDESD,720,0.011029,0.010286,0.009771,0.008171,0.009911
6,BONDESD,1080,0.014482,0.014429,0.015946,0.016893,0.016875
7,BONDESD,1440,0.006685,0.006542,0.007895,0.008295,0.010390
8,BONDESD,1800,0.016512,0.016405,0.014375,0.015093,0.017351
9,BONDESF,1,0.112500,0.112500,0.112500,0.112500,0.112500


,TIPO,PLAZO,FECHA_MAX_DELTA_1,FECHA_MAX_DELTA_2,FECHA_MAX_DELTA_3,FECHA_MAX_DELTA_4,FECHA_MAX_DELTA_5
0,BONDESD,1,2024-01-04,2024-01-05,2024-01-08,2024-01-09,2024-01-17
1,BONDESD,28,2024-02-16,2024-02-19,2024-02-20,2024-02-21,2024-02-29
2,BONDESD,91,2024-04-30,2024-05-02,2024-05-03,2024-05-06,2024-05-07
3,BONDESD,182,2024-02-29,2024-05-02,2024-05-02,2024-05-02,2024-05-07
4,BONDESD,360,2022-05-17,2022-05-18,2022-05-17,2022-05-17,2022-05-17
5,BONDESD,720,2022-11-30,2022-11-30,2022-11-30,2022-12-05,2024-04-29
6,BONDESD,1080,2021-12-14,2021-12-14,2025-01-16,2025-01-16,2025-01-16
7,BONDESD,1440,2021-12-14,2021-12-14,2024-01-25,2024-01-23,2024-01-25
8,BONDESD,1800,2024-06-14,2024-06-17,2024-06-18,2023-01-10,2023-01-11
9,BONDESF,1,2023-12-21,2023-12-22,2023-12-26,2023-12-27,2023-12-28


,TIPO,PLAZO,DELTA_1,DELTA_2,DELTA_3,DELTA_4,DELTA_5,FECHA_MAX_DELTA_1,FECHA_MAX_DELTA_2,FECHA_MAX_DELTA_3,FECHA_MAX_DELTA_4,FECHA_MAX_DELTA_5
0,BONDESD,1,0.115000,0.115000,0.115000,0.115000,0.115000,2024-01-04,2024-01-05,2024-01-08,2024-01-09,2024-01-17
1,BONDESD,28,0.117500,0.117500,0.117500,0.117500,0.116500,2024-02-16,2024-02-19,2024-02-20,2024-02-21,2024-02-29
2,BONDESD,91,0.012381,0.012619,0.012500,0.012143,0.014310,2024-04-30,2024-05-02,2024-05-03,2024-05-06,2024-05-07
3,BONDESD,182,0.008316,0.010000,0.010000,0.010000,0.014000,2024-02-29,2024-05-02,2024-05-02,2024-05-02,2024-05-07
4,BONDESD,360,0.109829,0.108922,0.108425,0.108075,0.107724,2022-05-17,2022-05-18,2022-05-17,2022-05-17,2022-05-17
5,BONDESD,720,0.011029,0.010286,0.009771,0.008171,0.009911,2022-11-30,2022-11-30,2022-11-30,2022-12-05,2024-04-29
6,BONDESD,1080,0.014482,0.014429,0.015946,0.016893,0.016875,2021-12-14,2021-12-14,2025-01-16,2025-01-16,2025-01-16
7,BONDESD,1440,0.006685,0.006542,0.007895,0.008295,0.010390,2021-12-14,2021-12-14,2024-01-25,2024-01-23,2024-01-25
8,BONDESD,1800,0.016512,0.016405,0.014375,0.015093,0.017351,2024-06-14,2024-06-17,2024-06-18,2023-01-10,2023-01-11
9,BONDESF,1,0.112500,0.112500,0.112500,0.112500,0.112500,2023-12-21,2023-12-22,2023-12-26,2023-12-27,2023-12-28


## Tasas

* TIIE 28 DÍAS
* TIIE fondeo a 1 DÍA
* CETES 28 DÍAS
* SOFR

In [12]:
cetes_28dr = API_Banxico(series_tiief1d, fecha_inicio, fecha_fin).dropna(how = 'all')
tiie_28dr = API_Banxico(series_tiie28d, fecha_inicio, fecha_fin).dropna(how = 'all')
tiief_1dr = API_Banxico(series_cetes28d, fecha_inicio, fecha_fin).dropna(how = 'all')

# Homologación nombres y formato de columnas
sofrr = pd.read_csv(r'C:\Users\pmendoza\Downloads\SOFR.csv', usecols = ['Effective Date', 'Rate (%)'], encoding = \
                    'latin1').rename(columns = {'Effective Date': 'FECHA', 'Rate (%)': 'TASA_INTERES'}).dropna(how = 'all')
sofrr['FECHA'] = pd.to_datetime(sofrr['FECHA'])

# Se considera unicamente el periodo de tiempo con el cual se cuenta informaición diaria (sin NA)
tiief_1d = periodo_continuo(tiief_1dr, 'TASA_INTERES')
cetes_28d = periodo_continuo(cetes_28dr, 'TASA_INTERES')
tiie_28d = periodo_continuo(tiie_28dr, 'TASA_INTERES')
sofr = periodo_continuo(sofrr, 'TASA_INTERES')

# Se eliminan filas completas de vacios
sofr.dropna(how = 'all', inplace = True)

# Se agrega como nueva clasificación el tipo de tasa
cetes_28d['TIPO'] = 'CETES28'
tiie_28d['TIPO'] = 'TIIE28'
tiief_1d['TIPO'] = 'TIIEF1D'
sofr['TIPO'] = 'SOFR'

# Se agrupa en un mismo df todas las tasas de interés
tasas = pd.concat([cetes_28d, tiie_28d, tiief_1d, sofr])

# Conversión a sistema decimal
tasas['TASA_INTERES'] = tasas['TASA_INTERES']/100
tasas

C:\Users\pmendoza\AppData\Local\Temp\ipykernel_11968\3167287423.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tiie_28d['TIPO'] = 'TIIE28'
C:\Users\pmendoza\AppData\Local\Temp\ipykernel_11968\3167287423.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tiief_1d['TIPO'] = 'TIIEF1D'


,FECHA,TASA_INTERES,TIPO
0,2006-01-02,0.0827,CETES28
1,2006-01-03,0.0828,CETES28
2,2006-01-04,0.0828,CETES28
3,2006-01-05,0.0828,CETES28
4,2006-01-06,0.0828,CETES28
...,...,...,...
1698,2018-04-06,0.0175,SOFR
1699,2018-04-05,0.0175,SOFR
1700,2018-04-04,0.0174,SOFR
1701,2018-04-03,0.0183,SOFR


In [13]:
ts = []

# Se aplica la variación diaria de tasa a tiempo t - tasa a tiwmpo t -i desde i = 1 hasta i = 5
# para cada plazo y tipo sobretasa
for (tasa), grupo in tasas.groupby('TIPO'):
    ts.append(delta_i_rate(grupo.copy(), col_tasa = 'TASA_INTERES'))    
ts = pd.concat(ts)
display(ts)

# Se obtienen las máximas variaciones diarias
delta_columns = [col for col in ts.columns if 'DELTA' in col]
ts_max = ts.groupby('TIPO')[delta_columns].max().reset_index()
display(ts_max)

# Se obtienen las fechas correspondientes a las máximas variaciones diarias
ts_max_fechas = ts.groupby('TIPO').apply(
    lambda grupo: pd.Series({
        f'FECHA_MAX_{col}': grupo.loc[grupo[col].idxmax(), 'FECHA'] if not grupo[col].isna().all() else None
        for col in delta_columns
    })).reset_index()
display(ts_max_fechas)

# Se crea un resumen con las máximas variaciones diarias, para cada tasa con sus respectivas fechas
maximos_tasas = ts_max.merge(ts_max_fechas, on = 'TIPO')
maximos_tasas

,FECHA,TASA_INTERES,TIPO,DELTA_1,DELTA_2,DELTA_3,DELTA_4,DELTA_5
0,2006-01-02,0.0827,CETES28,NaN,NaN,NaN,NaN,NaN
1,2006-01-03,0.0828,CETES28,-0.0001,NaN,NaN,NaN,NaN
2,2006-01-04,0.0828,CETES28,0.0000,-0.0001,NaN,NaN,NaN
3,2006-01-05,0.0828,CETES28,0.0000,0.0000,-0.0001,NaN,NaN
4,2006-01-06,0.0828,CETES28,0.0000,0.0000,0.0000,-0.0001,NaN
...,...,...,...,...,...,...,...,...
1512,2024-12-26,0.0974,TIIEF1D,0.0006,0.0021,0.0016,0.0021,0.0031
1513,2025-01-02,0.1004,TIIEF1D,-0.0030,-0.0024,-0.0009,-0.0014,-0.0009
1514,2025-01-09,0.0988,TIIEF1D,0.0016,-0.0014,-0.0008,0.0007,0.0002
1515,2025-01-16,0.0978,TIIEF1D,0.0010,0.0026,-0.0004,0.0002,0.0017


,TIPO,DELTA_1,DELTA_2,DELTA_3,DELTA_4,DELTA_5
0,CETES28,0.0079,0.0100,0.0155,0.01100,0.01100
1,SOFR,0.0270,0.0330,0.0339,0.03400,0.03290
2,TIIE28,0.0587,0.0540,0.0864,0.08235,0.10885
3,TIIEF1D,0.1026,0.1341,0.1109,0.11310,0.14440


,TIPO,FECHA_MAX_DELTA_1,FECHA_MAX_DELTA_2,FECHA_MAX_DELTA_3,FECHA_MAX_DELTA_4,FECHA_MAX_DELTA_5
0,CETES28,2009-03-20,2008-10-24,2008-10-24,2008-10-27,2008-10-24
1,SOFR,2019-09-18,2019-09-19,2019-09-20,2019-09-23,2019-09-24
2,TIIE28,1998-09-17,1998-09-18,1998-09-21,1998-09-22,1998-09-23
3,TIIEF1D,1998-09-24,1998-10-01,1998-10-08,1998-10-15,1998-10-22


,TIPO,DELTA_1,DELTA_2,DELTA_3,DELTA_4,DELTA_5,FECHA_MAX_DELTA_1,FECHA_MAX_DELTA_2,FECHA_MAX_DELTA_3,FECHA_MAX_DELTA_4,FECHA_MAX_DELTA_5
0,CETES28,0.0079,0.0100,0.0155,0.01100,0.01100,2009-03-20,2008-10-24,2008-10-24,2008-10-27,2008-10-24
1,SOFR,0.0270,0.0330,0.0339,0.03400,0.03290,2019-09-18,2019-09-19,2019-09-20,2019-09-23,2019-09-24
2,TIIE28,0.0587,0.0540,0.0864,0.08235,0.10885,1998-09-17,1998-09-18,1998-09-21,1998-09-22,1998-09-23
3,TIIEF1D,0.1026,0.1341,0.1109,0.11310,0.14440,1998-09-24,1998-10-01,1998-10-08,1998-10-15,1998-10-22


## Determinación de los puntos críticos

#### Definición de umbrales y consolidación de información de tasas y sobretasas

In [14]:
umbrales_tasas = {'0.03': 'darkorange', '0.02': 'gold', '0.01': 'moccasin'}
umbrales_sobretasas = {'0.4': 'darkorange', '0.3': 'gold', '0.2': 'moccasin'}

In [17]:
inicio_hist_sbts = sobretasas.groupby('TIPO')['FECHA'].agg(['min', 'max']).reset_index().\
                                rename(columns = {'min': 'INICIO', 'max': 'FIN'})
inicio_hist_ts = tasas.groupby('TIPO')['FECHA'].agg(['min', 'max']).reset_index().\
                                rename(columns = {'min': 'INICIO', 'max': 'FIN'})

TYS_max = pd.concat([sbts_max, ts_max])
TYS = pd.concat([sbts.rename(columns = {'SOBRETASA': 'VALOR'}), ts.rename(columns = {'TASA_INTERES': 'VALOR'})])
maximos = pd.concat([maximos_sobretasas, maximos_tasas]).reset_index(drop = True)
inicio_historicos = pd.concat([inicio_hist_sbts, inicio_hist_ts])

In [21]:
#Se definen las columnas delta
delta_cols = [col for col in maximos.columns if col.startswith('DELTA_') and 'FECHA_MAX_' not in col]
fecha_max_cols = [col for col in maximos.columns if col.startswith('FECHA_MAX_DELTA_')]

In [22]:
matriz_M_tasas = aplicar_resaltado(maximos_tasas, delta_cols, umbrales_tasas).\
                            format(formato_condicional_umbrales(maximos_tasas))
display(matriz_M_tasas)


matriz_M_sobretasas = aplicar_resaltado(maximos_sobretasas, delta_cols, umbrales_sobretasas)\
                                .format(formato_condicional_umbrales(maximos_sobretasas))
display(matriz_M_sobretasas)


umbrales_dict = {'maximos_tasas': umbrales_tasas, 'maximos_sobretasas': umbrales_sobretasas}
# matriz_M = aplicar_resaltado(maximos, delta_cols, {**umbrales_dict['maximos_tasas'], 
#                                                    **umbrales_dict['maximos_sobretasas']}).\
#                                                     format(formato_condicional_umbrales(maximos))
# matriz_M

,TIPO,DELTA_1,DELTA_2,DELTA_3,DELTA_4,DELTA_5,FECHA_MAX_DELTA_1,FECHA_MAX_DELTA_2,FECHA_MAX_DELTA_3,FECHA_MAX_DELTA_4,FECHA_MAX_DELTA_5
0,CETES28,0.79%,1.00%,1.55%,1.10%,1.10%,2009-03-20,2008-10-24,2008-10-24,2008-10-27,2008-10-24
1,SOFR,2.70%,3.30%,3.39%,3.40%,3.29%,2019-09-18,2019-09-19,2019-09-20,2019-09-23,2019-09-24
2,TIIE28,5.87%,5.40%,8.64%,8.24%,10.88%,1998-09-17,1998-09-18,1998-09-21,1998-09-22,1998-09-23
3,TIIEF1D,10.26%,13.41%,11.09%,11.31%,14.44%,1998-09-24,1998-10-01,1998-10-08,1998-10-15,1998-10-22


,TIPO,PLAZO,DELTA_1,DELTA_2,DELTA_3,DELTA_4,DELTA_5,FECHA_MAX_DELTA_1,FECHA_MAX_DELTA_2,FECHA_MAX_DELTA_3,FECHA_MAX_DELTA_4,FECHA_MAX_DELTA_5
0,BONDESD,1,11.50%,11.50%,11.50%,11.50%,11.50%,2024-01-04,2024-01-05,2024-01-08,2024-01-09,2024-01-17
1,BONDESD,28,11.75%,11.75%,11.75%,11.75%,11.65%,2024-02-16,2024-02-19,2024-02-20,2024-02-21,2024-02-29
2,BONDESD,91,1.24%,1.26%,1.25%,1.21%,1.43%,2024-04-30,2024-05-02,2024-05-03,2024-05-06,2024-05-07
3,BONDESD,182,0.83%,1.00%,1.00%,1.00%,1.40%,2024-02-29,2024-05-02,2024-05-02,2024-05-02,2024-05-07
4,BONDESD,360,10.98%,10.89%,10.84%,10.81%,10.77%,2022-05-17,2022-05-18,2022-05-17,2022-05-17,2022-05-17
5,BONDESD,720,1.10%,1.03%,0.98%,0.82%,0.99%,2022-11-30,2022-11-30,2022-11-30,2022-12-05,2024-04-29
6,BONDESD,1080,1.45%,1.44%,1.59%,1.69%,1.69%,2021-12-14,2021-12-14,2025-01-16,2025-01-16,2025-01-16
7,BONDESD,1440,0.67%,0.65%,0.79%,0.83%,1.04%,2021-12-14,2021-12-14,2024-01-25,2024-01-23,2024-01-25
8,BONDESD,1800,1.65%,1.64%,1.44%,1.51%,1.74%,2024-06-14,2024-06-17,2024-06-18,2023-01-10,2023-01-11
9,BONDESF,1,11.25%,11.25%,11.25%,11.25%,11.25%,2023-12-21,2023-12-22,2023-12-26,2023-12-27,2023-12-28


## Exportar Resultados

In [23]:
# Consolidado tasas y sobretasas
with pd.ExcelWriter('ANALISIS (CONSOLIDADO) HISTORICO DE TASAS Y SOBRETASAS.xlsx', engine = "openpyxl") as writer:
    inicio_historicos.to_excel(writer, sheet_name = 'PERIODO HISTORICO', index = False)
    matriz_M_tasas.to_excel(writer, sheet_name = 'TASAS - VAR MAX', index = False)
    matriz_M_sobretasas.to_excel(writer, sheet_name = 'SOBRETASAS - VAR MAX', index = False)
#     matriz_M.to_excel(writer, sheet_name = 'MAX VAR y FECHAS', index = False)
    TYS.to_excel(writer, sheet_name = 'HISTORICO Y DELTAS', index = False)

In [24]:
t_fin = time.time()

In [25]:
minutos = (t_fin - t_inicio)/60
print(f'El tiempo código tardo {minutos} minutos en ejecutarse')

El tiempo código tardo 2.4536100069681805 minutos en ejecutarse
